# csv 데이터 불러오기

In [5]:
import pandas as pd

input_file = pd.read_csv('train_data0307.csv')
target_file = pd.read_csv('target_data0307.csv')

# dynamic 추출 함수

In [1]:
def distribute_dynamic(mean_velocity):
    """
    velocity에 따른 dynamic 기호 추출
    """
    dynamic = ''
    if 0 <= mean_velocity < 36:
        dynamic = 'ppp'
    elif 36 <= mean_velocity < 49:
        dynamic = 'pp'
    elif 49 <= mean_velocity < 62:
        dynamic = 'p'
    elif 62 <= mean_velocity < 75:
        dynamic = 'mp'
    elif 75 <= mean_velocity < 88:
        dynamic = 'mf'
    elif 88 <= mean_velocity < 101:
        dynamic = 'f'
    elif 101 <= mean_velocity < 114:
        dynamic = 'ff'
    elif 114 <= mean_velocity <= 127:
        dynamic = 'fff'
    
    return dynamic

# velocity에 대한 new_dynamic 컬럼 추가 함수

In [2]:
def add_new_dynamic_to_dataframe(input_file, target_file):
    """
    비어있는 dynamic 기호를 추가한다.
    1. note_on에 대한 dynamic 기호만 반영함.
        - dynamic 추가 방법
        1. 피아노 0.1초마다 note_on_status 값을 추출한다.
        2. 0.1 초에 해당하는 note_on 항목 중 velocity의 최대 값에 기반해 기호를 추출함.
    """
    input_file_note_status = [[{'note': x}, {'status': False}, {'velocity' : 0}] for x in range(128)]
    target_file_note_status = [[{'note': x}, {'status': False}, {'velocity' : 0}] for x in range(128)]

    input_dynamic_list = []
    target_dynamic_list = []
    
    # 인풋 데이터에 대해서 수행.
    for index in range(len(input_file)):

            target_temp_data = []

            notes_input = eval(input_file.at[index, 'note']) # 문자열을 -> 숫자로
            msg_type_input = eval(input_file.at[index, 'msg_type']) # 문자열을 -> 숫자로
            velocity = eval(input_file.at[index, 'velocity'])

            temp = list(zip(notes_input, msg_type_input, velocity)) # [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정

            # 피아노 활성화 값 추적 
            for data in temp:
                if data[1] == 'note_on':
                    input_file_note_status[data[0]][1]['status'] = True
                    input_file_note_status[data[0]][2]['velocity'] = data[2]
                elif data[1] == 'note_off':
                    input_file_note_status[data[0]][1]['status'] = False
                    input_file_note_status[data[0]][2]['velocity'] = data[2]

            velocity_list = []

            for on_data in input_file_note_status:
                if on_data[1]['status'] == True:
                    velocity_list.append(({'note': on_data[0]['note']}, {'velocity' : on_data[2]['velocity']}))

            max_velocity = 0 # 최대 값 찾기

            for velocity in velocity_list:
                # 최대값 찾기
                if max_velocity < velocity[1]['velocity']:
                    max_velocity = velocity[1]['velocity']
            dynamic = distribute_dynamic(max_velocity)
            input_dynamic_list.append(dynamic)
            
    
    # 타겟 데이터에 대해 수행
    for index in range(len(target_file)):


            notes_input = eval(target_file.at[index, 'note']) # 문자열을 -> 숫자로
            msg_type_input = eval(target_file.at[index, 'msg_type']) # 문자열을 -> 숫자로
            velocity = eval(target_file.at[index, 'velocity'])

            temp = list(zip(notes_input, msg_type_input, velocity)) # [(60, 'note_on'), (60, 'note_on')] 이와 같이 데이터 수정

            # 피아노 활성화 값 추적 
            for data in temp:
                if data[1] == 'note_on':
                    target_file_note_status[data[0]][1]['status'] = True
                    target_file_note_status[data[0]][2]['velocity'] = data[2]
                elif data[1] == 'note_off':
                    target_file_note_status[data[0]][1]['status'] = False
                    target_file_note_status[data[0]][2]['velocity'] = data[2]

            velocity_list = []

            for on_data in target_file_note_status:
                if on_data[1]['status'] == True:
                    velocity_list.append(({'note': on_data[0]['note']}, {'velocity' : on_data[2]['velocity']}))

            max_velocity = 0 # 최대 값 찾기

            for velocity in velocity_list:
                # 최대값 찾기
                if max_velocity < velocity[1]['velocity']:
                    max_velocity = velocity[1]['velocity']
            dynamic = distribute_dynamic(max_velocity)
            target_dynamic_list.append(dynamic)


    target_file.insert(loc=5, column='new_dynamic', value=target_dynamic_list)
    input_file.insert(loc=5, column='new_dynamic', value=input_dynamic_list)

In [12]:
add_new_dynamic_to_dataframe(input_file, target_file)

# 비교 알고리즘

In [13]:
input_file.head()

,sec,msg_type,channel,note,velocity,new_dynamic,dynamic,accent,count,main_vol,depth,pedal,pan
0,0.0,[],[],[],[],ppp,ppp,0,0,100,0,0,0
1,0.1,"['note_on', 'note_on']","[0, 0]","[60, 60]","[21, 21]",ppp,ppp,0,2,0,0,0,0
2,0.2,[],[],[],[],ppp,NaN,0,0,0,0,0,0
3,0.3,"['note_on', 'note_on']","[0, 0]","[64, 64]","[32, 32]",ppp,ppp,0,2,0,0,0,0
4,0.4,[],[],[],[],ppp,NaN,0,0,0,0,0,0


In [14]:
target_file.head()

,sec,msg_type,channel,note,velocity,new_dynamic,dynamic,accent,count,main_vol,depth,pedal,pan
0,0.0,['note_on'],[0],[60],[61],p,p,0,1,"[100, 100, 100]","[127, 127]",127,"[64, 64]"
1,0.1,[],[],[],[],p,NaN,0,0,0,0,0,0
2,0.2,['note_on'],[0],[64],[57],p,p,0,1,0,0,0,0
3,0.3,[],[],[],[],p,NaN,0,0,0,0,0,0
4,0.4,['note_on'],[0],[67],[56],p,p,0,1,0,0,0,0


## 기존 셈유림 유사도 알고리즘

In [15]:
# 최종 코드

def calculate_similarity_dynamic(input_df, target_df):

    input_counts = input_df['new_dynamic'].value_counts().to_dict()
    target_counts = target_df['new_dynamic'].value_counts().to_dict()
    
    # 공통된 'Dynamic' 값들의 유사도 계산
    common_keys = set(input_counts) & set(target_counts)

    # 총 가능한 차이의 합 및 실제 차이의 합 계산
    total_possible_difference = sum(max(input_counts.get(key, 0), target_counts.get(key, 0)) for key in common_keys)
    actual_difference = sum(abs(input_counts.get(key, 0) - target_counts.get(key, 0)) for key in common_keys)

    
    # ((총 가능한 차이의 합 - 실제 차이의 합) / 총 가능한 차이의 합) * 100
    similarity_percentage = ((total_possible_difference - actual_difference) / total_possible_difference) * 100
    return f'{similarity_percentage:.2f}%'

In [16]:
dynamic_similarity = calculate_similarity_dynamic(input_file, target_file)
print(f'셈여림 유사도: {dynamic_similarity}')

셈여림 유사도: 35.87%
